# AutoML - Regression

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup)  
* [Compare Models](#compare)  
* [Create Model](#create)  
* [Tune Model](#tune)  
* [Evaluate Model](#evaluate)  
* [Finalize and Store Model](#finalize_and_store)


## Imports and Global Settings

In [1]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pycaret.regression import RegressionExperiment

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import (
    ModelSetup,
    evaluate_reg_model,
    calculate_roi,
    save_model_report,
)

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Load Data

In [2]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [3]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [4]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [5]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df = games_features.copy()

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 3955
Data columns (total 715 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3956 non-null   object        
 1    game_datetime                                 3956 non-null   datetime64[ns]
 2    home_team                                     3956 non-null   object        
 3    away_team                                     3956 non-null   object        
 4    open_line                                     3853 non-null   float64       
 5    home_score                                    3855 non-null   float64       
 6    away_score                                    3855 non-null   float64       
 7    game_completed                                3955 non-null   object        
 8    odds_last_update                              101 non-nu

In [8]:
df.head(10)

,game_id,game_datetime,home_team,away_team,open_line,home_score,away_score,game_completed,odds_last_update,scores_last_update,season,elo1_pre,elo2_pre,elo_prob1,elo_prob2,last_5_hv,streak_hv,win_pct_hv,raptor1_pre,raptor2_pre,season_type,raptor_prob1,raptor_prob2,rest_diff_hv,carm_elo1_pre,carm_elo2_pre,day_of_season,point_diff_hv,carm_elo_prob1,carm_elo_prob2,away_team_last_5,away_team_streak,home_team_last_5,home_team_streak,away_team_win_pct,home_team_win_pct,point_diff_last_5_hv,pie_away_all_advanced,pie_away_l2w_advanced,pie_home_all_advanced,pie_home_l2w_advanced,l_away_all_traditional,l_away_l2w_traditional,l_home_all_traditional,l_home_l2w_traditional,pace_away_all_advanced,pace_away_l2w_advanced,pace_home_all_advanced,pace_home_l2w_advanced,poss_away_all_advanced,poss_away_l2w_advanced,poss_home_all_advanced,poss_home_l2w_advanced,w_away_all_traditional,w_away_l2w_traditional,w_home_all_traditional,w_home_l2w_traditional,gp_away_all_traditional,gp_away_l2w_traditional,gp_home_all_traditional,gp_home_l2w_traditional,ast_to_away_all_advanced,ast_to_away_l2w_advanced,ast_to_home_all_advanced,ast_to_home_l2w_advanced,away_team_avg_point_diff,e_pace_away_all_advanced,e_pace_away_l2w_advanced,e_pace_home_all_advanced,e_pace_home_l2w_advanced,home_team_avg_point_diff,min_away_all_traditional,min_away_l2w_traditional,min_home_all_traditional,min_home_l2w_traditional,opp_pf_away_all_opponent,opp_pf_away_l2w_opponent,opp_pf_home_all_opponent,opp_pf_home_l2w_opponent,pts_away_all_traditional,pts_away_l2w_traditional,pts_home_all_traditional,pts_home_l2w_traditional,ts_pct_away_all_advanced,ts_pct_away_l2w_advanced,ts_pct_home_all_advanced,ts_pct_home_l2w_advanced,ast_pct_away_all_advanced,ast_pct_away_l2w_advanced,ast_pct_home_all_advanced,ast_pct_home_l2w_advanced,days_since_last_game_away,days_since_last_game_home,efg_pct_away_all_advanced,efg_pct_away_l2w_advanced,efg_pct_home_all_advanced,efg_pct_home_l2w_advanced,opp_ast_away_all_opponent,opp_ast_away_l2w_opponent,opp_ast_home_all_opponent,opp_ast_home_l2w_opponent,opp_blk_away_all_opponent,opp_blk_away_l2w_opponent,opp_blk_home_all_opponent,opp_blk_home_l2w_opponent,opp_fga_away_all_opponent,opp_fga_away_l2w_opponent,opp_fga_home_all_opponent,opp_fga_home_l2w_opponent,opp_fgm_away_all_opponent,opp_fgm_away_l2w_opponent,opp_fgm_home_all_opponent,opp_fgm_home_l2w_opponent,opp_fta_away_all_opponent,opp_fta_away_l2w_opponent,opp_fta_home_all_opponent,opp_fta_home_l2w_opponent,opp_ftm_away_all_opponent,opp_ftm_away_l2w_opponent,opp_ftm_home_all_opponent,opp_ftm_home_l2w_opponent,opp_pfd_away_all_opponent,opp_pfd_away_l2w_opponent,opp_pfd_home_all_opponent,opp_pfd_home_l2w_opponent,opp_pts_away_all_opponent,opp_pts_away_l2w_opponent,opp_pts_home_all_opponent,opp_pts_home_l2w_opponent,opp_reb_away_all_opponent,opp_reb_away_l2w_opponent,opp_reb_home_all_opponent,opp_reb_home_l2w_opponent,opp_stl_away_all_opponent,opp_stl_away_l2w_opponent,opp_stl_home_all_opponent,opp_stl_home_l2w_opponent,opp_tov_away_all_opponent,opp_tov_away_l2w_opponent,opp_tov_home_all_opponent,opp_tov_home_l2w_opponent,reb_pct_away_all_advanced,reb_pct_away_l2w_advanced,reb_pct_home_all_advanced,reb_pct_home_l2w_advanced,dreb_pct_away_all_advanced,dreb_pct_away_l2w_advanced,dreb_pct_home_all_advanced,dreb_pct_home_l2w_advanced,opp_blka_away_all_opponent,opp_blka_away_l2w_opponent,opp_blka_home_all_opponent,opp_blka_home_l2w_opponent,opp_dreb_away_all_opponent,opp_dreb_away_l2w_opponent,opp_dreb_home_all_opponent,opp_dreb_home_l2w_opponent,opp_fg3a_away_all_opponent,opp_fg3a_away_l2w_opponent,opp_fg3a_home_all_opponent,opp_fg3a_home_l2w_opponent,opp_fg3m_away_all_opponent,opp_fg3m_away_l2w_opponent,opp_fg3m_home_all_opponent,opp_fg3m_home_l2w_opponent,opp_oreb_away_all_opponent,opp_oreb_away_l2w_opponent,opp_oreb_home_all_opponent,opp_oreb_home_l2w_opponent,oreb_pct_away_all_advanced,oreb_pct_away_l2w_advanced,oreb_pct_home_all_advanced,oreb_pct_home_l2w_advanced,w_pct_away_all_traditional,w_pct_away_l2w_trad

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [9]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

### Create Targets

In [10]:
df = ModelSetup.add_targets(df)

### Select Features

In [11]:
training_dates, testing_dates = ModelSetup.choose_dates([2020, 2021], [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2020-12-22', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [12]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
elo1_pre
elo2_pre
elo_prob1
elo_prob2
last_5_hv
streak_hv
win_pct_hv
raptor1_pre
raptor2_pre
season_type
raptor_prob1
raptor_prob2
rest_diff_hv
carm_elo1_pre
carm_elo2_pre
day_of_season
point_diff_hv
carm_elo_prob1
carm_elo_prob2
away_team_last_5
away_team_streak
home_team_last_5
home_team_streak
away_team_win_pct
home_team_win_pct
point_diff_last_5_hv
pie_away_all_advanced
pie_away_l2w_advanced
pie_home_all_advanced
pie_home_l2w_advanced
l_away_all_traditional
l_away_l2w_traditional
l_home_all_traditional
l_home_l2w_traditional
pace_away_all_advanced
pace_away_l2w_advanced
pace_home_all_advanced
pace_home_l2w_advanced
poss_away_all_advanced
poss_away_l2w_advanced
poss_home_all_advanced
poss_home_l2w_advanced
w_away_all_traditional
w_away_l2w_traditional
w_home_all_traditional
w_home_l2w_traditional
gp_away_all_traditional
gp_away_l2w_traditional
gp_home_a

In [13]:
features_to_use = ["open_line"]

In [14]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "reg", features_to_use, training_dates, testing_dates, create_report=True
)

In [15]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2399, 4)
Testing Shape:  (1230, 4)


### Baselines

In [16]:
training_baseline_via_vegas = model_report["ind_baseline_train"]
testing_baseline_via_vegas = model_report["ind_baseline_test"]

training_baseline_via_mean = model_report["dep_baseline_train"]
testing_baseline_via_mean = model_report["dep_baseline_test"]

print(f"Training Baseline via Vegas: {training_baseline_via_vegas:.2f}")
print(f"Testing Baseline via Vegas: {testing_baseline_via_vegas:.2f}")
print(f"Training Baseline via Mean: {training_baseline_via_mean:.2f}")
print(f"Testing Baseline via Mean: {testing_baseline_via_mean:.2f}")

Training Baseline via Vegas: 10.69
Testing Baseline via Vegas: 9.76
Training Baseline via Mean: 12.18
Testing Baseline via Mean: 10.97


<a id=Regression></a>

## Regression

In [17]:
py_reg = RegressionExperiment()

<a id=setup></a>

### Setup and Preprocessing

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [18]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

setup_params_reg = {
    "log_experiment": True,
    "log_profile": False,
    "log_plots": False,
    "experiment_name": f"REG_1_{timestamp}",
    "data": training_df,
    "test_data": testing_df,
    "target": "REG_TARGET",
    "preprocess": False,
    "normalize": False,  # zscore
    "transformation": False,  # yeo-johnson power transform to make data more Gaussian
    "remove_outliers": False,  # using SVD
    "remove_multicollinearity": False,
    "feature_selection": False,
    "pca": False,
    "pca_components": 10,
    "numeric_features": [],
    "ignore_features": ["game_id", "vegas_open_hv"],
}

In [19]:
py_reg.setup(**setup_params_reg)

,Description,Value
0,Session id,4753
1,Target,REG_TARGET
2,Target type,Regression
3,Original data shape,"(3629, 4)"
4,Transformed data shape,"(3629, 2)"
5,Transformed train set shape,"(2399, 2)"
6,Transformed test set shape,"(1230, 2)"
7,Ignore features,2
8,Numeric features,1


2023/08/20 20:07:15 INFO mlflow.tracking.fluent: Experiment with name 'REG_1_20230820200709' does not exist. Creating a new experiment.


<a id=compare></a>

### Compare Models

In [20]:
best_model_reg = py_reg.compare_models(turbo=False, sort="MAE")

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
svm,Support Vector Regression,10.6839,186.6168,13.6430,0.1922,0.9586,1.1690,0.3510
en,Elastic Net,10.6891,185.2486,13.5954,0.1974,1.0168,1.1647,0.1680
lasso,Lasso Regression,10.6892,185.2496,13.5954,0.1974,1.0167,1.1647,0.1250
llar,Lasso Least Angle Regression,10.6892,185.2496,13.5954,0.1974,1.0167,1.1647,0.1550
br,Bayesian Ridge,10.6892,185.2228,13.5945,0.1975,1.0101,1.1743,0.1740
ard,Automatic Relevance Determination,10.6892,185.2228,13.5945,0.1975,1.0101,1.1743,0.5000
ridge,Ridge Regression,10.6893,185.2221,13.5945,0.1975,1.0095,1.1752,0.1460
lr,Linear Regression,10.6893,185.2221,13.5945,0.1975,1.0095,1.1752,0.5650
omp,Orthogonal Matching Pursuit,10.6893,185.2221,13.5945,0.1975,1.0095,1.1752,0.1750
lar,Least Angle Regression,10.6893,185.2221,13.5945,0.1975,1.0095,1.1752,0.2240


Processing:   0%|          | 0/109 [00:00<?, ?it/s]

In [21]:
print(best_model_reg)

SVR()


<a id=create></a>

### Create Selected Model

In [22]:
model_reg = py_reg.create_model("svm")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11.4942,208.4009,14.4361,0.1029,0.9427,1.3646
1,10.3173,160.7374,12.6782,0.1761,0.9590,1.2239
2,11.2559,215.0382,14.6642,0.1982,1.0306,1.1321
3,10.1276,162.3114,12.7401,0.2173,0.9123,1.1160
4,10.6928,191.1500,13.8257,0.2388,0.9493,1.0777
5,9.8499,159.7209,12.6381,0.1944,0.9681,1.0596
6,10.9595,191.5759,13.8411,0.1222,0.9968,1.3199
7,10.6521,194.3859,13.9422,0.1541,0.9883,1.1137
8,10.3532,182.9938,13.5275,0.2612,0.9379,1.2135


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

<a id=tune></a>

### Tune Selected Model

In [23]:
tuned_model_reg = py_reg.tune_model(model_reg)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11.5196,208.9860,14.4563,0.1004,0.9488,1.3794
1,10.3205,161.3459,12.7022,0.1730,0.9561,1.2208
2,11.2072,214.5095,14.6461,0.2002,1.0245,1.1396
3,10.1350,161.9622,12.7264,0.2190,0.9082,1.1366
4,10.6812,189.7937,13.7766,0.2442,0.9495,1.0927
5,9.8669,159.9062,12.6454,0.1934,0.9810,1.0645
6,10.9955,192.2535,13.8656,0.1191,0.9729,1.3268
7,10.6714,194.8382,13.9584,0.1522,0.9875,1.1233
8,10.3548,181.4860,13.4717,0.2673,0.9524,1.2276


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [24]:
print(tuned_model_reg)

SVR(C=7.149, epsilon=1.2)


In [25]:
model_report["details"] = tuned_model_reg.get_params()

<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [26]:
py_reg.evaluate_model(tuned_model_reg)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [27]:
# py_reg.interpret_model(tuned_model_reg)

In [28]:
train_predictions_reg = py_reg.predict_model(tuned_model_reg, data=training_df)
train_prediction_metrics = py_reg.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Support Vector Regression,10.6412,185.1679,13.6076,0.2026,0.9523,1.1775


In [29]:
model_report["train_mae"] = train_prediction_metrics["MAE"][0]
model_report["train_r2"] = train_prediction_metrics["R2"][0]

In [30]:
test_predictions_reg = py_reg.predict_model(tuned_model_reg, data=testing_df)
test_prediction_metrics = py_reg.pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Support Vector Regression,9.8404,158.9116,12.6060,0.1488,0.9280,1.2445


In [31]:
model_report["test_mae"] = test_prediction_metrics["MAE"][0]
model_report["test_r2"] = test_prediction_metrics["R2"][0]

In [32]:
train_acc_reg, train_closer_to_target_reg, train_prediction_df_reg = evaluate_reg_model(
    train_predictions_reg, "vegas_open_hv", "REG_TARGET", "prediction_label"
)
test_acc_reg, test_closer_to_target_reg, test_prediction_df_reg = evaluate_reg_model(
    test_predictions_reg, "vegas_open_hv", "REG_TARGET", "prediction_label"
)

Prediction is closer to target in 52.15% of cases
Accuracy: 0.5261
Prediction is closer to target in 47.40% of cases
Accuracy: 0.4829


In [33]:
model_report["train_acc_reg"] = train_acc_reg
model_report["test_acc_reg"] = test_acc_reg
model_report["train_ctt"] = train_closer_to_target_reg
model_report["test_ctt"] = test_closer_to_target_reg

In [34]:
roi_results_reg = calculate_roi(test_prediction_df_reg, "actual_side", "pred_side")
roi_results_reg

,Label,Total ROI,Average ROI per Bet
0,"All Bets, Even Amount",-4200,-3.41
1,"All Bets, Typical Odds",-9546,-7.76


In [35]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]
model_report["roi_all_bets_typical_odds_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [36]:
final_model_reg = py_reg.finalize_model(tuned_model_reg)

In [37]:
platform = "pycaret"
problem_type = "reg"
model_type = "svm"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

'pycaret_reg_svm_2023_08_20_20_07_01'

In [38]:
py_reg.save_model(final_model_reg, f"../models/AutoML/{model_id}")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('placeholder', None),
                 ('actual_estimator', SVR(C=7.149, epsilon=1.2))]),
 '../models/AutoML/pycaret_reg_svm_2023_08_20_20_07_01.pkl')

In [39]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [40]:
model_report

{'datetime': Timestamp('2023-08-20 20:07:01.667014'),
 'problem_type': 'reg',
 'target': 'REG_TARGET',
 'features': ['open_line'],
 'training_start_date': Timestamp('2020-12-22 00:00:00'),
 'training_end_date': Timestamp('2022-04-10 00:00:00'),
 'testing_start_date': Timestamp('2022-10-18 00:00:00'),
 'testing_end_date': Timestamp('2023-04-09 00:00:00'),
 'training_df_shape': (2399, 4),
 'testing_df_shape': (1230, 4),
 'ind_baseline_train': 10.688620258441016,
 'ind_baseline_test': 9.759756097560976,
 'dep_baseline_train': 12.182451664155606,
 'dep_baseline_test': 10.969119585735248,
 'details': {'C': 7.149,
  'cache_size': 200,
  'coef0': 0.0,
  'degree': 3,
  'epsilon': 1.2,
  'gamma': 'scale',
  'kernel': 'rbf',
  'max_iter': -1,
  'shrinking': True,
  'tol': 0.001,
  'verbose': False},
 'train_mae': 10.6412,
 'train_r2': 0.2026,
 'test_mae': 9.8404,
 'test_r2': 0.1488,
 'train_acc_reg': 0.5260525218841183,
 'test_acc_reg': 0.48292682926829267,
 'train_ctt': 0.5214672780325136,
 'te

In [41]:
save_model_report(model_report)